# Parte III - Importação das Libs - Features Selection

In [152]:
import numpy as np
import sqlite3 as sql
from tqdm import tqdm
import glob
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import feature_engine

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import RFECV, f_classif, chi2

from scipy.stats import chi2_contingency
from scipy.stats import pointbiserialr
!pip install boruta
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from boruta import BorutaPy
from sklearn.metrics import roc_auc_score

from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


In [153]:
# CARREGANDO O DATAFRAME
Queimadas_Pantanal = pd.read_csv('Queimadas_Pantanal_EDA.csv')

In [154]:
Queimadas_Pantanal.shape

(2562, 13)

# Criação variável target

In [155]:
conditions = [
    (Queimadas_Pantanal['RISCO_FOGO'] <= 0.7),
    (Queimadas_Pantanal['RISCO_FOGO'] > 0.7)
    ]

In [156]:
values = [False, True]

In [157]:
Queimadas_Pantanal['ALERTA_FOGO'] = np.select(conditions, values)

In [158]:
Queimadas_Pantanal['ALERTA_FOGO'].unique()

array([1, 0])

In [160]:
# Conferência
Queimadas_Pantanal[Queimadas_Pantanal['ALERTA_FOGO']== True]

,DATE,DIAS_SEM_CHUVA,RISCO_FOGO,PRECIPITACAO_DIARIA,PRESSAO_ATMOSFERICA,TEMPERATURA_PONTO_DE_ORVALHO,TEMPERATURA_MAXIMA,TEMPERATURA_MEDIA,TEMPERATURA_MINIMA,UMIDADE_RELATIVA_DO_AR_MEDIA,UMIDADE_RELATIVA_MINIMA_DO_AR,VENTO_RAJADA_MAXIMA,VENTO_VELOCIDADE_MEDIA,ALERTA_FOGO
0,2014-01-03,2.0,0.8,0.0,995.71,21.03,32.7,26.46,21.6,74.21,47.0,5.1,0.65,1
4,2014-01-10,3.0,0.9,0.0,997.34,21.91,36.4,31.60,27.6,57.42,36.0,6.6,1.48,1
14,2014-01-31,4.0,1.0,0.0,998.38,20.65,34.4,28.00,23.3,65.63,42.0,5.6,1.16,1
17,2014-02-05,3.0,0.9,20.6,999.14,21.00,25.4,23.33,18.5,86.92,78.0,8.8,2.58,1
20,2014-02-08,0.0,1.0,0.0,1000.30,19.38,30.4,26.97,24.4,63.58,50.0,5.3,1.94,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,2020-11-27,7.0,1.0,0.0,994.00,19.90,39.4,31.89,28.2,51.13,27.0,13.4,1.17,1
2548,2020-12-14,2.0,1.0,4.2,994.95,20.75,35.9,0.00,23.8,63.05,38.0,8.5,0.65,1
2551,2020-12-22,4.0,0.8,0.0,995.33,20.75,35.7,30.38,26.4,57.25,40.0,10.9,1.65,1
2552,2020-12-25,0.0,0.9,0.2,993.11,22.61,36.5,30.07,25.2,66.46,40.0,6.3,0.10,1


In [161]:
# Dropando varíavel Risco Fogo

Queimadas_Pantanal.drop('RISCO_FOGO', axis=1, inplace=True)

In [162]:
Queimadas_Pantanal.columns

Index(['DATE', 'DIAS_SEM_CHUVA', 'PRECIPITACAO_DIARIA', 'PRESSAO_ATMOSFERICA',
       'TEMPERATURA_PONTO_DE_ORVALHO', 'TEMPERATURA_MAXIMA',
       'TEMPERATURA_MEDIA', 'TEMPERATURA_MINIMA',
       'UMIDADE_RELATIVA_DO_AR_MEDIA', 'UMIDADE_RELATIVA_MINIMA_DO_AR',
       'VENTO_RAJADA_MAXIMA', 'VENTO_VELOCIDADE_MEDIA', 'ALERTA_FOGO'],
      dtype='object')

## Criação variável Mês

In [163]:
Queimadas_Pantanal['DATE'] = pd.to_datetime(Queimadas_Pantanal['DATE'])
Queimadas_Pantanal['MONTH'] = Queimadas_Pantanal['DATE'].dt.month

In [164]:
Queimadas_Pantanal.drop('DATE', axis=1, inplace=True)

In [165]:
Queimadas_Pantanal.columns

Index(['DIAS_SEM_CHUVA', 'PRECIPITACAO_DIARIA', 'PRESSAO_ATMOSFERICA',
       'TEMPERATURA_PONTO_DE_ORVALHO', 'TEMPERATURA_MAXIMA',
       'TEMPERATURA_MEDIA', 'TEMPERATURA_MINIMA',
       'UMIDADE_RELATIVA_DO_AR_MEDIA', 'UMIDADE_RELATIVA_MINIMA_DO_AR',
       'VENTO_RAJADA_MAXIMA', 'VENTO_VELOCIDADE_MEDIA', 'ALERTA_FOGO',
       'MONTH'],
      dtype='object')

# Split Treino e Teste

In [166]:
# SPLIT DO DATAFRAME EM TREINO E TESTE
df_train, df_test = train_test_split(Queimadas_Pantanal, test_size=0.2, random_state=42)

In [167]:
X_train = df_train.drop('ALERTA_FOGO', axis=1)
y_train = df_train['ALERTA_FOGO']

In [168]:
df_train.shape, df_test.shape

((2049, 13), (513, 13))

In [169]:
y_train

1134    0
1223    0
1927    1
2163    0
707     1
       ..
1638    0
1095    1
1130    1
1294    1
860     0
Name: ALERTA_FOGO, Length: 2049, dtype: int32

In [170]:
target = ['ALERTA_FOGO']
num_cols = ['DIAS_SEM_CHUVA','PRECIPITACAO_DIARIA','PRESSAO_ATMOSFERICA','TEMPERATURA_PONTO_DE_ORVALHO','TEMPERATURA_MAXIMA','TEMPERATURA_MEDIA','TEMPERATURA_MINIMA','UMIDADE_RELATIVA_DO_AR_MEDIA','UMIDADE_RELATIVA_MINIMA_DO_AR','VENTO_RAJADA_MAXIMA','VENTO_VELOCIDADE_MEDIA', 'MONTH']

# ANOVA

In [171]:
X_train_num = X_train[num_cols]

In [172]:
anova_result = f_classif(X_train_num, y_train)

In [173]:
p_values_num_features = pd.Series(anova_result[1])
p_values_num_features.index = num_cols

In [174]:
filter_num_features = p_values_num_features[p_values_num_features < 0.05].index.tolist()
filter_num_features

['DIAS_SEM_CHUVA',
 'PRESSAO_ATMOSFERICA',
 'TEMPERATURA_PONTO_DE_ORVALHO',
 'TEMPERATURA_MEDIA',
 'TEMPERATURA_MINIMA',
 'UMIDADE_RELATIVA_DO_AR_MEDIA',
 'UMIDADE_RELATIVA_MINIMA_DO_AR',
 'VENTO_VELOCIDADE_MEDIA',
 'MONTH']

# Recursive Feature Elimination

In [175]:
# 1a Selação (Random Forest)
forest = RandomForestRegressor(n_jobs = -1, max_depth = 5)

rfecv_RFC = RFECV(estimator=forest, scoring='neg_mean_squared_error')
rfecv_RFC.fit(X_train,y_train)
print(rfecv_RFC.n_features_) # Número de features selecionadas como as mais importantes 
print(rfecv_RFC.support_) # Vetor de booleans. True se a variável deve ser mantida e False caso contrário
mask_RFC = rfecv_RFC.support_


10
[ True False  True  True False  True  True  True  True  True  True  True]


# BORUTA

In [176]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder, StandardScaler

In [177]:
# Inicializa o encoder
from feature_engine.encoding import CountFrequencyEncoder

# fenc = CountFrequencyEncoder(encoding_method='frequency', variables=num_cols)
fenc = preprocessing.LabelEncoder()
standard_scaler = StandardScaler()

# Aplica a transformacao em X_train
y_train_encoded = fenc.fit_transform(y_train)
X_train_encoded = standard_scaler.fit_transform(X_train)

In [178]:
# Iniacializa algoritmo Random Forest para utilizacao no Boruta
forest = RandomForestClassifier(n_jobs=-1, max_depth=5)
# Inicializa o Boruta
boruta_selector = BorutaPy(forest, n_estimators=50, max_iter=100, random_state=42)

In [179]:
X_train.head()

,DIAS_SEM_CHUVA,PRECIPITACAO_DIARIA,PRESSAO_ATMOSFERICA,TEMPERATURA_PONTO_DE_ORVALHO,TEMPERATURA_MAXIMA,TEMPERATURA_MEDIA,TEMPERATURA_MINIMA,UMIDADE_RELATIVA_DO_AR_MEDIA,UMIDADE_RELATIVA_MINIMA_DO_AR,VENTO_RAJADA_MAXIMA,VENTO_VELOCIDADE_MEDIA,MONTH
1134,1.0,0.0,993.30,20.46,38.7,30.39,26.3,57.17,29.0,11.9,1.73,11
1223,13.0,0.0,997.49,19.38,32.7,27.45,22.6,62.46,45.0,5.1,1.03,4
1927,0.0,0.0,996.46,23.14,31.6,27.00,23.9,79.83,58.0,8.6,1.43,6
2163,0.0,0.0,0.00,0.00,29.8,0.00,22.0,70.58,58.0,0.0,0.00,6
707,3.0,0.0,996.79,22.65,33.5,27.79,24.6,74.42,50.0,13.0,1.45,3


In [180]:
y_train

1134    0
1223    0
1927    1
2163    0
707     1
       ..
1638    0
1095    1
1130    1
1294    1
860     0
Name: ALERTA_FOGO, Length: 2049, dtype: int32

In [181]:
# Ajusta o Boruta sobre os dados de treino
boruta_selector.fit(np.array(X_train), y_train)


BorutaPy(estimator=RandomForestClassifier(max_depth=5, n_estimators=50,
                                          n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x20B4CFCD640),
         n_estimators=50, random_state=RandomState(MT19937) at 0x20B4CFCD640)

In [182]:
# Visualiza variáveis significativas
boruta_selector.support_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [183]:
# Visualiza variáveis com significancia duvidosa
boruta_selector.support_weak_

array([False, False, False, False, False, False, False, False, False,
       False, False, False])

In [184]:
# De acordo com os resultados de feature selection, todas as variáveis precisam ser mantidas, tendo em vista que o anova e
# boruta consideram todas as variáveis significativas, apesar da RFECV ter rejeita a coluna de UMIDADE_RELATIVA_DO_AR_MEDIA.

In [185]:
#Exportação arquivo csv

In [186]:
Queimadas_Pantanal.to_csv('Queimadas_Pantanal_FS.csv', index = False)

# DATA PREP - Rever

In [78]:
# # Define step para features numéricas
# pipe_num_features = (
#     'robust_scaler',
#     RobustScaler(),
#     num_cols_updated
# )

In [79]:
# # Cria ColumnTransformer para preprocessamento
# transformers = [pipe_cat_features, pipe_num_features]
# pipe_transformers = ColumnTransformer(transformers)